In [3]:
from core.connection import get_session, save_on_database, get_from_database
from core.config import settings
from pyspark.sql import SparkSession
from pyspark.sql import DataFrame
import pyspark.sql.functions as F
import pyspark.sql.types as T

spark: SparkSession = get_session()

## 🥉 Bronze Layer

### Load data from csv

In [4]:
data: DataFrame = spark.read.csv(
    f"{settings.BASE_DIR / 'data_csv' / 'survey_results_public.csv'}",
    sep=",",
    header=True,
    inferSchema=True,
)

data.show(5)

24/06/24 20:58:36 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----------+-----+----------+--------------+--------------+------------------+--------------------+--------------------+--------------------+--------------------+----------------+---------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+----------+----------+----------+----------+----------+----------+----------+----------+-----------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+---------------+----------------+----------------+---------------------+---------------------+---------------------+---------------------+---------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+--------------------+--------------------+------+----------+---------------+--------------+--------------------+-------------------+--------------------+--------

### Select only data used in Data Warehouse

In [5]:
data = data.select(
    "Respondent",
    "Hobby",
    "OpenSource",
    "ConvertedSalary",
    "CommunicationTools",
    "LanguageWorkedWith",
    "OperatingSystem",
    "CompanySize",
    "Country",
)

data.show(5)

+----------+-----+----------+---------------+--------------------+--------------------+---------------+--------------------+--------------+
|Respondent|Hobby|OpenSource|ConvertedSalary|  CommunicationTools|  LanguageWorkedWith|OperatingSystem|         CompanySize|       Country|
+----------+-----+----------+---------------+--------------------+--------------------+---------------+--------------------+--------------+
|         1|  Yes|        No|             NA|               Slack|JavaScript;Python...|    Linux-based|  20 to 99 employees|         Kenya|
|         3|  Yes|       Yes|          70841|Confluence;Office...|JavaScript;Python...|    Linux-based|10,000 or more em...|United Kingdom|
|         4|  Yes|       Yes|             NA|                  NA|                  NA|             NA|  20 to 99 employees| United States|
|         5|   No|        No|             NA|                  NA|C#;JavaScript;SQL...|        Windows|100 to 499 employees| United States|
|         7|  Yes|  

## 🥈 Silver Layer

### Replace Na to None

In [6]:
data = data.replace("NA", None)

### Add id column

In [34]:
data = data.withColumn("id", F.col("Respondent"))
data.show(5)

+----------+-----+----------+---------------+--------------------+--------------------+---------------+--------------------+--------------+---+
|Respondent|Hobby|OpenSource|ConvertedSalary|  CommunicationTools|  LanguageWorkedWith|OperatingSystem|         CompanySize|       Country| id|
+----------+-----+----------+---------------+--------------------+--------------------+---------------+--------------------+--------------+---+
|         1|  Yes|        No|           NULL|               Slack|JavaScript;Python...|    Linux-based|  20 to 99 employees|         Kenya|  1|
|         3|  Yes|       Yes|          70841|Confluence;Office...|JavaScript;Python...|    Linux-based|10,000 or more em...|United Kingdom|  3|
|         4|  Yes|       Yes|           NULL|                NULL|                NULL|           NULL|  20 to 99 employees| United States|  4|
|         5|   No|        No|           NULL|                NULL|C#;JavaScript;SQL...|        Windows|100 to 499 employees| United Stat

### Change respondent name as described in the challenge.


In [35]:
@F.udf(returnType=T.StringType())
def _add_name_prefix(x):
    return f"respondent_{x}"


data = data.withColumn("name", _add_name_prefix(F.col("Respondent"))).drop(
    "Respondent"
)
data.show(5)

+-----+----------+---------------+--------------------+--------------------+---------------+--------------------+--------------+---+------------+
|Hobby|OpenSource|ConvertedSalary|  CommunicationTools|  LanguageWorkedWith|OperatingSystem|         CompanySize|       Country| id|        name|
+-----+----------+---------------+--------------------+--------------------+---------------+--------------------+--------------+---+------------+
|  Yes|        No|           NULL|               Slack|JavaScript;Python...|    Linux-based|  20 to 99 employees|         Kenya|  1|respondent_1|
|  Yes|       Yes|          70841|Confluence;Office...|JavaScript;Python...|    Linux-based|10,000 or more em...|United Kingdom|  3|respondent_3|
|  Yes|       Yes|           NULL|                NULL|                NULL|           NULL|  20 to 99 employees| United States|  4|respondent_4|
|   No|        No|           NULL|                NULL|C#;JavaScript;SQL...|        Windows|100 to 499 employees| United Sta

### Insert data in operation_system

In [36]:
operating_system = (
    data.select(F.col("OperatingSystem").alias("name")).distinct().dropna()
)

save_on_database(operating_system, "operation_system")

### Insert data in country

In [37]:
country = data.select(F.col("Country").alias("name")).distinct().dropna()

save_on_database(country, "country")

### Insert data in company

In [38]:
company = data.select(F.col("CompanySize").alias("size")).distinct().dropna()
save_on_database(company, "company")

## Insert data in respondent

#### Create function to reference id from database & replace value

In [39]:
def get_reference_from_database(
    spark_session: SparkSession, table_name: str
) -> dict[str, int]:
    try:
        df: DataFrame = get_from_database(spark_session, table_name)
    except Exception:
        print("Connection Error")

    return {v: k for k, v in dict(df.collect()).items()}


def replace_value(x, to_replace: dict):
    return to_replace.get(x, T.NullType())

### create column company_id

In [40]:
company_db = get_reference_from_database(spark, "company")


@F.udf(returnType=T.IntegerType())
def company_replace_udf(x):
    return replace_value(x, company_db)


data = data.withColumn(
    "company_id", company_replace_udf(F.col("CompanySize"))
).drop("CompanySize")
data.show(5)

+-----+----------+---------------+--------------------+--------------------+---------------+--------------+---+------------+----------+
|Hobby|OpenSource|ConvertedSalary|  CommunicationTools|  LanguageWorkedWith|OperatingSystem|       Country| id|        name|company_id|
+-----+----------+---------------+--------------------+--------------------+---------------+--------------+---+------------+----------+
|  Yes|        No|           NULL|               Slack|JavaScript;Python...|    Linux-based|         Kenya|  1|respondent_1|         5|
|  Yes|       Yes|          70841|Confluence;Office...|JavaScript;Python...|    Linux-based|United Kingdom|  3|respondent_3|         7|
|  Yes|       Yes|           NULL|                NULL|                NULL|           NULL| United States|  4|respondent_4|         5|
|   No|        No|           NULL|                NULL|C#;JavaScript;SQL...|        Windows| United States|  5|respondent_5|         2|
|  Yes|        No|          21426|Office / produ

#### create column operation_system_id

In [41]:
os_db = get_reference_from_database(spark, "operation_system")


@F.udf(returnType=T.IntegerType())
def os_replace_udf(x):
    return replace_value(x, os_db)


data = data.withColumn(
    "operation_system_id", os_replace_udf(F.col("OperatingSystem"))
).drop("OperatingSystem")
data.show(5)

+-----+----------+---------------+--------------------+--------------------+--------------+---+------------+----------+-------------------+
|Hobby|OpenSource|ConvertedSalary|  CommunicationTools|  LanguageWorkedWith|       Country| id|        name|company_id|operation_system_id|
+-----+----------+---------------+--------------------+--------------------+--------------+---+------------+----------+-------------------+
|  Yes|        No|           NULL|               Slack|JavaScript;Python...|         Kenya|  1|respondent_1|         5|                  1|
|  Yes|       Yes|          70841|Confluence;Office...|JavaScript;Python...|United Kingdom|  3|respondent_3|         7|                  1|
|  Yes|       Yes|           NULL|                NULL|                NULL| United States|  4|respondent_4|         5|               NULL|
|   No|        No|           NULL|                NULL|C#;JavaScript;SQL...| United States|  5|respondent_5|         2|                  4|
|  Yes|        No|  

#### create column country_id

In [42]:
country_db = get_reference_from_database(spark, "country")


@F.udf(returnType=T.IntegerType())
def country_replace_udf(x):
    return replace_value(x, country_db)


data = data.withColumn(
    "country_id", country_replace_udf(F.col("Country"))
).drop("Country")

data.show(5)

+-----+----------+---------------+--------------------+--------------------+---+------------+----------+-------------------+----------+
|Hobby|OpenSource|ConvertedSalary|  CommunicationTools|  LanguageWorkedWith| id|        name|company_id|operation_system_id|country_id|
+-----+----------+---------------+--------------------+--------------------+---+------------+----------+-------------------+----------+
|  Yes|        No|           NULL|               Slack|JavaScript;Python...|  1|respondent_1|         5|                  1|        97|
|  Yes|       Yes|          70841|Confluence;Office...|JavaScript;Python...|  3|respondent_3|         7|                  1|       127|
|  Yes|       Yes|           NULL|                NULL|                NULL|  4|respondent_4|         5|               NULL|        38|
|   No|        No|           NULL|                NULL|C#;JavaScript;SQL...|  5|respondent_5|         2|                  4|        38|
|  Yes|        No|          21426|Office / produ

### Change ConvertedSalary to Decimal and transform null to 0.0

In [43]:
data = data.withColumn(
    "ConvertedSalary", data["ConvertedSalary"].cast(T.DoubleType())
).fillna(0.0, subset="ConvertedSalary")
data.show()

+-----+----------+---------------+--------------------+--------------------+---+-------------+----------+-------------------+----------+
|Hobby|OpenSource|ConvertedSalary|  CommunicationTools|  LanguageWorkedWith| id|         name|company_id|operation_system_id|country_id|
+-----+----------+---------------+--------------------+--------------------+---+-------------+----------+-------------------+----------+
|  Yes|        No|            0.0|               Slack|JavaScript;Python...|  1| respondent_1|         5|                  1|        97|
|  Yes|       Yes|        70841.0|Confluence;Office...|JavaScript;Python...|  3| respondent_3|         7|                  1|       127|
|  Yes|       Yes|            0.0|                NULL|                NULL|  4| respondent_4|         5|               NULL|        38|
|   No|        No|            0.0|                NULL|C#;JavaScript;SQL...|  5| respondent_5|         2|                  4|        38|
|  Yes|        No|        21426.0|Office 

#### Transform salary

In [44]:
@F.udf(returnType=T.DoubleType())
def convert_salary(x):
    return round((x / settings.YEAR_MONTHS) * settings.DOLLAR_EXCHANGE, 2)


data = data.withColumn(
    "salary", convert_salary(F.col("ConvertedSalary"))
).drop("ConvertedSalary")
data.show(5)

+-----+----------+--------------------+--------------------+---+------------+----------+-------------------+----------+--------+
|Hobby|OpenSource|  CommunicationTools|  LanguageWorkedWith| id|        name|company_id|operation_system_id|country_id|  salary|
+-----+----------+--------------------+--------------------+---+------------+----------+-------------------+----------+--------+
|  Yes|        No|               Slack|JavaScript;Python...|  1|respondent_1|         5|                  1|        97|     0.0|
|  Yes|       Yes|Confluence;Office...|JavaScript;Python...|  3|respondent_3|         7|                  1|       127|22492.02|
|  Yes|       Yes|                NULL|                NULL|  4|respondent_4|         5|               NULL|        38|     0.0|
|   No|        No|                NULL|C#;JavaScript;SQL...|  5|respondent_5|         2|                  4|        38|     0.0|
|  Yes|        No|Office / producti...|C;C++;Java;Matlab...|  7|respondent_7|         7|         

#### create respondent table

In [45]:
respondent = (
    data.select(
        "id",
        "name",
        "OpenSource",
        "Hobby",
        "salary",
        "operation_system_id",
        "country_id",
        "company_id",
    )
    .withColumns(
        {
            "open_source": F.col("OpenSource").cast(T.BooleanType()),
            "hobby": F.col("Hobby").cast(T.BooleanType()),
        }
    )
    .drop("OpenSource")
)
respondent.show(5)

+---+------------+-----+--------+-------------------+----------+----------+-----------+
| id|        name|hobby|  salary|operation_system_id|country_id|company_id|open_source|
+---+------------+-----+--------+-------------------+----------+----------+-----------+
|  1|respondent_1| true|     0.0|                  1|        97|         5|      false|
|  3|respondent_3| true|22492.02|                  1|       127|         7|       true|
|  4|respondent_4| true|     0.0|               NULL|        38|         5|       true|
|  5|respondent_5|false|     0.0|                  4|        38|         2|      false|
|  7|respondent_7| true| 6802.76|                  4|       121|         7|      false|
+---+------------+-----+--------+-------------------+----------+----------+-----------+
only showing top 5 rows



#### Insert data in respondent

In [46]:
save_on_database(respondent, "respondent")

### Create function to separate values in CommunicationTools and LanguageWorkedWith

In [47]:
def sep_col_values(df: DataFrame, col: str, sep: str = ";") -> DataFrame:

    col_separated = (
        data.select((F.split(col, sep))).rdd.flatMap(lambda x: x).collect()
    )

    col_separated = set(
        [
            item
            for sublist in col_separated
            if sublist is not None
            for item in sublist
        ]
    )

    return spark.createDataFrame(
        col_separated,
        schema=T.StringType(),
    ).withColumnRenamed("value", "name")

### Insert data in programming_language

In [48]:
programming_language = sep_col_values(data, "LanguageWorkedWith")
save_on_database(programming_language, "programming_language")
programming_language.show(5)

+------+
|  name|
+------+
|   SQL|
|   C++|
|VB.NET|
| Cobol|
|    C#|
+------+
only showing top 5 rows



### Insert data in communications_tools

In [49]:
communications_tools = sep_col_values(data, "CommunicationTools")
save_on_database(communications_tools, "communications_tools")
communications_tools.show(5)

+--------------------+
|                name|
+--------------------+
|               Slack|
|              Trello|
|Other wiki tool (...|
|            Facebook|
|Other chat system...|
+--------------------+
only showing top 5 rows



### Create function to separate every language in row

In [50]:
def col_list_to_df(df: DataFrame, col: str, sep: str = ";") -> DataFrame:

    data_rdd = df.select("id", F.split(col, ";")).dropna().rdd.collect()

    data_list = [list(zip([b] * len(c), c)) for b, c in data_rdd]

    split_data = [y for x in data_list for y in x]

    return spark.createDataFrame(
        split_data,
        T.StructType(
            [
                T.StructField("respondent_id", T.LongType()),
                T.StructField("name", T.StringType()),
            ]
        ),
    )

#### Create dataframe resp_programming_language

In [51]:
programming_language_ref = get_reference_from_database(
    spark, "programming_language"
)

resp_programming_language = col_list_to_df(data, "LanguageWorkedWith")


@F.udf(returnType=T.IntegerType())
def programming_replace_udf(x):
    return replace_value(x, programming_language_ref)


resp_programming_language = resp_programming_language.withColumn(
    "programming_language_id", programming_replace_udf(F.col("name"))
).drop("name")

resp_programming_language.show(5)

+-------------+-----------------------+
|respondent_id|programming_language_id|
+-------------+-----------------------+
|            1|                     14|
|            1|                     15|
|            1|                      7|
|            1|                     16|
|            3|                     14|
+-------------+-----------------------+
only showing top 5 rows



#### Create dataframe resp_tools

In [52]:
communications_tools_ref = get_reference_from_database(
    spark, "communications_tools"
)

resp_tools = col_list_to_df(data, "CommunicationTools")


@F.udf(returnType=T.IntegerType())
def communication_replace_udf(x):
    return replace_value(x, communications_tools_ref)


resp_tools = resp_tools.withColumn(
    "communications_tools_id", communication_replace_udf(F.col("name"))
).drop("name")

resp_tools.show(5)

+-------------+-----------------------+
|respondent_id|communications_tools_id|
+-------------+-----------------------+
|            1|                      6|
|            3|                      1|
|            3|                      8|
|            3|                      6|
|            3|                      3|
+-------------+-----------------------+
only showing top 5 rows



### Insert data in resp_programming_language

In [53]:
save_on_database(resp_programming_language, "resp_programming_language")

### Insert data in resp_tools

In [54]:
save_on_database(resp_tools, "resp_tools")